In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import preprocessing, model_selection
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import random
optimizers = tf.keras.optimizers.SGD()

# Bispectrum 2D Flat

In [2]:
def get_dataset_bispectrum2d_flat(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64)
                # print(rwb)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)
    # print(data)      
    return data

In [3]:
def get_batch_bispectrum2d_flat(path):
    # loading extracted feature & label
    x = get_dataset_bispectrum2d_flat(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

## Model

In [4]:
def create_model_CNN_bispectrum2d_flat(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,feature)))
    model.add(layers.Reshape((61, feature, 1)))

    model.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu"))
    if feature >= 9:
        model.add(layers.MaxPooling2D(pool_size=(3,3)))
    elif feature == 4:
        model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [5]:
def create_model_ANN_bispectrum2d_flat(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,feature)))
    model.add(layers.Reshape((61, feature, 1)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

# Bispectrum 2D

In [6]:
def get_dataset_bispectrum2d(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64)
                # print(rwb)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)
    # print(data)      
    return data

In [7]:
def get_batch_bispectrum2d(path):
    # loading extracted feature & label
    x = get_dataset_bispectrum2d(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

## Model

In [8]:
def create_model_CNN_bispectrum2d(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,feature,feature)))

    print(feature)
    if feature >= 3:
        model.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu"))
    elif feature <=2:
        model.add(layers.Conv2D(filters=16, kernel_size=(2,2), activation="relu"))

    if feature >= 5:
        model.add(layers.MaxPooling2D(pool_size=(3,3)))
    elif feature == 4:
        model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [9]:
def create_model_ANN_bispectrum2d(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,feature,feature)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

# RWB

In [10]:
def get_dataset_rwb(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64).reshape(-1,1)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)      
    return data

In [11]:
def get_batch_rwb(path):
    # loading extracted feature & label
    x = get_dataset_rwb(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

## Model 

In [12]:
def create_model_CNN_rwb():
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(366,)))
    model.add(layers.Reshape((366, 1)))

    model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
    model.add(layers.MaxPooling1D(pool_size=3))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [13]:
def create_model_ANN_rwb():
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(366,)))
    model.add(layers.Reshape((366, 1)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

# RWB 2D

In [14]:
def get_dataset_rwb2d(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)      
    return data

In [15]:
def get_batch_rwb2d(path):
    # loading extracted feature & label
    x = get_dataset_rwb2d(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

## Model

In [16]:
def create_model_CNN_rwb_2d():
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,6)))
    model.add(layers.Reshape((61, 6, 1)))

    model.add(layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=3))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [17]:
def create_model_ANN_rwb_2d():
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(61,6)))
    model.add(layers.Reshape((61, 6, 1)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

# Bispectrum

In [18]:
def get_dataset_bispectrum(directory):
    data = pd.DataFrame(columns=['data', 'label'])
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label ='alcoholic'
                else:
                    label = 'control'

                temp_data = pd.DataFrame(columns=['data', 'label'], index=[0])

                rwb = np.load(rel_path)
                rwb.astype(np.float64).reshape(-1,1)
                # print(rwb)
                # with open(rel_path, 'r') as file:
                    
                #     rwb = list(csv.reader(file, delimiter=","))[0]
                #     # scaler = preprocessing.MinMaxScaler()
                #     rwb = np.asarray(rwb).astype(np.float64).reshape(-1,1)
                #     # print(rwb)
                                
                temp_data['data'][0] = rwb
                temp_data['label'] = label
                
                # decomp = np.arange(0, 366)
                # plt.plot(decomp, df_data)
                # plt.xlabel('Dimension Number')
                # plt.ylabel('Wavelet Bispectrum Energy')
                # plt.show()
                data = pd.concat([data, temp_data], ignore_index=True)
    label_map = {"alcoholic": 1, "control": 0}
    data['label_map'] = data['label'].map(label_map)
    # print(data)      
    return data

In [19]:
def get_batch_bispectrum(path):
    # loading extracted feature & label
    x = get_dataset_bispectrum(path)

    scaler = preprocessing.MinMaxScaler()

    series_list = [
        i for i in x["data"]
    ]

    # series_list = series_list.reshape(-1, 366, 1)

    labels_list = [i for i in x["label_map"]]
        
    # y = keras.utils.to_categorical(y[0])
    dataset = tf.data.Dataset.from_tensor_slices((series_list,labels_list))
    dataset = dataset.shuffle(len(labels_list)).batch(32)

    return dataset

## Model

In [20]:
def create_model_CNN_bispectrum(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(feature,)))
    model.add(layers.Reshape((feature, 1)))

    model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
    model.add(layers.MaxPooling1D(pool_size=3))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [21]:
def create_model_ANN_bispectrum(feature):
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(feature,)))
    model.add(layers.Reshape((feature, 1)))

    model.add(layers.Flatten())

    model.add(layers.Dense(512, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation="relu"))

    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

# Callbacks

In [22]:
def myCallbacks(log_dir):
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='acc',
    patience=50,
    mode='max', verbose = 0)
    model_path = os.path.join(log_dir,'best_model.h5')
    mc = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_acc', mode='max', verbose=0, save_best_only=True)
    return [tensorboard_callback, early_stopping, mc]
    

# Training

In [23]:
lags = [256, 128, 64, 32, 16, 8]
folds = ['train_1', 'test_1', 'epoch_1', 'train_2', 'test_2', 'epoch_2']
time_measured = ['Wall_Time_1', 'CPU_Time_1', 'Wall_Time_2', 'CPU_Time_2']
epochs = 2000

## RWB 2D

### CNN

In [24]:
log_dirs = ['logs/CNN_RWB2D','logs1/CNN_RWB2D', 'logs2/CNN_RWB2D', 'logs3/CNN_RWB2D', 'logs4/CNN_RWB2D', 'logs5/CNN_RWB2D', 'logs6/CNN_RWB2D', 'logs7/CNN_RWB2D', 'logs8/CNN_RWB2D', 'logs9/CNN_RWB2D']
train_dir = '../Feature/RWB2D/Train/smni_cmi_train_rwb2d'
test_dir = '../Feature/RWB2D/Test/smni_cmi_test_rwb2d'
for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured)
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_rwb2d(train_temp_dir)
            test_ds = get_batch_rwb2d(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_CNN_rwb_2d()
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 61, 6, 1)          0         
                                                                 
 conv2d (Conv2D)             (None, 59, 4, 16)         160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 19, 1, 16)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 304)               0         
                                                                 
 dense (Dense)               (None, 512)               156160    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                        

### ANN

In [25]:
log_dirs = ['logs/ANN_RWB2D','logs1/ANN_RWB2D', 'logs2/ANN_RWB2D', 'logs3/ANN_RWB2D', 'logs4/ANN_RWB2D', 'logs5/ANN_RWB2D', 'logs6/ANN_RWB2D', 'logs7/ANN_RWB2D', 'logs8/ANN_RWB2D', 'logs9/ANN_RWB2D']
train_dir = '../Feature/RWB2D/Train/smni_cmi_train_rwb2d'
test_dir = '../Feature/RWB2D/Test/smni_cmi_test_rwb2d'
for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured)
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_rwb2d(train_temp_dir)
            test_ds = get_batch_rwb2d(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_ANN_rwb_2d()
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_120"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_120 (Reshape)       (None, 61, 6, 1)          0         
                                                                 
 flatten_120 (Flatten)       (None, 366)               0         
                                                                 
 dense_360 (Dense)           (None, 512)               187904    
                                                                 
 dropout_240 (Dropout)       (None, 512)               0         
                                                                 
 dense_361 (Dense)           (None, 256)               131328    
                                                                 
 dropout_241 (Dropout)       (None, 256)               0         
                                                                 
 dense_362 (Dense)           (None, 1)              

## Bispectrum

### CNN

In [26]:
log_dirs = ['logs/CNN_Bispectrum','logs1/CNN_Bispectrum', 'logs2/CNN_Bispectrum', 'logs3/CNN_Bispectrum', 'logs4/CNN_Bispectrum', 'logs5/CNN_Bispectrum', 'logs6/CNN_Bispectrum', 'logs7/CNN_Bispectrum', 'logs8/CNN_Bispectrum', 'logs9/CNN_Bispectrum']
train_dir = '../Feature/Bispectrum1D/Train/smni_cmi_train_bispectrum1d'
test_dir = '../Feature/Bispectrum1D/Test/smni_cmi_test_bispectrum1d'
for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured) 
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_bispectrum(train_temp_dir)
            test_ds = get_batch_bispectrum(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_CNN_bispectrum([i[0].shape[1] for i in train][0])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        print('test')
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))

Model: "sequential_240"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_240 (Reshape)       (None, 1525, 1)           0         
                                                                 
 conv1d (Conv1D)             (None, 1523, 16)          64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 507, 16)          0         
 )                                                               
                                                                 
 flatten_240 (Flatten)       (None, 8112)              0         
                                                                 
 dense_720 (Dense)           (None, 512)               4153856   
                                                                 
 dropout_480 (Dropout)       (None, 512)               0         
                                                    

### ANN

In [27]:
log_dirs = ['logs/ANN_Bispectrum','logs1/ANN_Bispectrum', 'logs2/ANN_Bispectrum', 'logs3/ANN_Bispectrum', 'logs4/ANN_Bispectrum', 'logs5/ANN_Bispectrum', 'logs6/ANN_Bispectrum', 'logs7/ANN_Bispectrum', 'logs8/ANN_Bispectrum', 'logs9/ANN_Bispectrum']
train_dir = '../Feature/Bispectrum1D/Train/smni_cmi_train_bispectrum1d'
test_dir = '../Feature/Bispectrum1D/Test/smni_cmi_test_bispectrum1d'
for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured) 
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_bispectrum(train_temp_dir)
            test_ds = get_batch_bispectrum(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_ANN_bispectrum([i[0].shape[1] for i in train][0])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        print('test')
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))

Model: "sequential_360"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_360 (Reshape)       (None, 1525, 1)           0         
                                                                 
 flatten_360 (Flatten)       (None, 1525)              0         
                                                                 
 dense_1080 (Dense)          (None, 512)               781312    
                                                                 
 dropout_720 (Dropout)       (None, 512)               0         
                                                                 
 dense_1081 (Dense)          (None, 256)               131328    
                                                                 
 dropout_721 (Dropout)       (None, 256)               0         
                                                                 
 dense_1082 (Dense)          (None, 1)              

## RWB

### CNN

In [28]:
log_dirs = ['logs/CNN_RWB','logs1/CNN_RWB', 'logs2/CNN_RWB', 'logs3/CNN_RWB', 'logs4/CNN_RWB', 'logs5/CNN_RWB', 'logs6/CNN_RWB', 'logs7/CNN_RWB', 'logs8/CNN_RWB', 'logs9/CNN_RWB']
train_dir = '../Feature/RWB/Train/smni_cmi_train_feature'
test_dir = '../Feature/RWB/Test/smni_cmi_test_feature'

for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured)
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_rwb(train_temp_dir)
            test_ds = get_batch_rwb(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_CNN_rwb()
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_480"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_480 (Reshape)       (None, 366, 1)            0         
                                                                 
 conv1d_120 (Conv1D)         (None, 364, 16)           64        
                                                                 
 max_pooling1d_120 (MaxPooli  (None, 121, 16)          0         
 ng1D)                                                           
                                                                 
 flatten_480 (Flatten)       (None, 1936)              0         
                                                                 
 dense_1440 (Dense)          (None, 512)               991744    
                                                                 
 dropout_960 (Dropout)       (None, 512)               0         
                                                    

### ANN

In [29]:
log_dirs = ['logs/ANN_RWB','logs1/ANN_RWB', 'logs2/ANN_RWB', 'logs3/ANN_RWB', 'logs4/ANN_RWB', 'logs5/ANN_RWB', 'logs6/ANN_RWB', 'logs7/ANN_RWB', 'logs8/ANN_RWB', 'logs9/ANN_RWB']
train_dir = '../Feature/RWB/Train/smni_cmi_train_feature'
test_dir = '../Feature/RWB/Test/smni_cmi_test_feature'

for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured)
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_rwb(train_temp_dir)
            test_ds = get_batch_rwb(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_ANN_rwb()
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_600"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_600 (Reshape)       (None, 366, 1)            0         
                                                                 
 flatten_600 (Flatten)       (None, 366)               0         
                                                                 
 dense_1800 (Dense)          (None, 512)               187904    
                                                                 
 dropout_1200 (Dropout)      (None, 512)               0         
                                                                 
 dense_1801 (Dense)          (None, 256)               131328    
                                                                 
 dropout_1201 (Dropout)      (None, 256)               0         
                                                                 
 dense_1802 (Dense)          (None, 1)              

## Bispectrum 2D

### CNN

In [30]:
log_dirs = ['logs/CNN_Bispec2D','logs1/CNN_Bispec2D', 'logs2/CNN_Bispec2D', 'logs3/CNN_Bispec2D', 'logs4/CNN_Bispec2D', 'logs5/CNN_Bispec2D', 'logs6/CNN_Bispec2D', 'logs7/CNN_Bispec2D', 'logs8/CNN_Bispec2D', 'logs9/CNN_Bispec2D']
train_dir = '../Feature/Bispectrum2D/Train/smni_cmi_train_bispectrum2d'
test_dir = '../Feature/Bispectrum2D/Test/smni_cmi_test_bispectrum2d'

for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured) 
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_bispectrum2d(train_temp_dir)
            test_ds = get_batch_bispectrum2d(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_CNN_bispectrum2d([i[0].shape[2] for i in train][0])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            print("Ini Lag Lag Lag = ",lag)

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        print('test')
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


5
Model: "sequential_720"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_120 (Conv2D)         (None, 59, 3, 16)         736       
                                                                 
 max_pooling2d_120 (MaxPooli  (None, 19, 1, 16)        0         
 ng2D)                                                           
                                                                 
 flatten_720 (Flatten)       (None, 304)               0         
                                                                 
 dense_2160 (Dense)          (None, 512)               156160    
                                                                 
 dropout_1440 (Dropout)      (None, 512)               0         
                                                                 
 dense_2161 (Dense)          (None, 256)               131328    
                                                  

### ANN

In [31]:
log_dirs = ['logs/ANN_Bispec2D','logs1/ANN_Bispec2D', 'logs2/ANN_Bispec2D', 'logs3/ANN_Bispec2D', 'logs4/ANN_Bispec2D', 'logs5/ANN_Bispec2D', 'logs6/ANN_Bispec2D', 'logs7/ANN_Bispec2D', 'logs8/ANN_Bispec2D', 'logs9/ANN_Bispec2D']
train_dir = '../Feature/Bispectrum2D/Train/smni_cmi_train_bispectrum2d'
test_dir = '../Feature/Bispectrum2D/Test/smni_cmi_test_bispectrum2d'

for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured) 
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_bispectrum2d(train_temp_dir)
            test_ds = get_batch_bispectrum2d(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_ANN_bispectrum2d([i[0].shape[2] for i in train][1])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        print('test')
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_840"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_840 (Flatten)       (None, 1525)              0         
                                                                 
 dense_2520 (Dense)          (None, 512)               781312    
                                                                 
 dropout_1680 (Dropout)      (None, 512)               0         
                                                                 
 dense_2521 (Dense)          (None, 256)               131328    
                                                                 
 dropout_1681 (Dropout)      (None, 256)               0         
                                                                 
 dense_2522 (Dense)          (None, 1)                 257       
                                                                 
Total params: 912,897
Trainable params: 912,897
Non-

## Bispectrum 2D Flat

### CNN

In [32]:
log_dirs = ['logs/CNN_Bispec2DFlat','logs1/CNN_Bispec2DFlat', 'logs2/CNN_Bispec2DFlat', 'logs3/CNN_Bispec2DFlat', 'logs4/CNN_Bispec2DFlat', 'logs5/CNN_Bispec2DFlat', 'logs6/CNN_Bispec2DFlat', 'logs7/CNN_Bispec2DFlat', 'logs8/CNN_Bispec2DFlat', 'logs9/CNN_Bispec2DFlat']
train_dir = '../Feature/Bispectrum2DFlat/Train/smni_cmi_train_bispectrum2dflat'
test_dir = '../Feature/Bispectrum2DFlat/Test/smni_cmi_test_bispectrum2dflat'

for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured) 
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_bispectrum2d_flat(train_temp_dir)
            test_ds = get_batch_bispectrum2d_flat(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_CNN_bispectrum2d_flat([i[0].shape[2] for i in train][0])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        print('test')
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_960"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_720 (Reshape)       (None, 61, 25, 1)         0         
                                                                 
 conv2d_240 (Conv2D)         (None, 59, 23, 16)        160       
                                                                 
 max_pooling2d_220 (MaxPooli  (None, 19, 7, 16)        0         
 ng2D)                                                           
                                                                 
 flatten_960 (Flatten)       (None, 2128)              0         
                                                                 
 dense_2880 (Dense)          (None, 512)               1090048   
                                                                 
 dropout_1920 (Dropout)      (None, 512)               0         
                                                    

### ANN

In [33]:
log_dirs = ['logs/ANN_Bispec2DFlat','logs1/ANN_Bispec2DFlat', 'logs2/ANN_Bispec2DFlat', 'logs3/ANN_Bispec2DFlat', 'logs4/ANN_Bispec2DFlat', 'logs5/ANN_Bispec2DFlat', 'logs6/ANN_Bispec2DFlat', 'logs7/ANN_Bispec2DFlat', 'logs8/ANN_Bispec2DFlat', 'logs9/ANN_Bispec2DFlat']
train_dir = '../Feature/Bispectrum2DFlat/Train/smni_cmi_train_bispectrum2dflat'
test_dir = '../Feature/Bispectrum2DFlat/Test/smni_cmi_test_bispectrum2dflat'

for log_dir in log_dirs:
    recap = pd.DataFrame(index=lags, columns=folds)
    training_time = pd.DataFrame(index=lags, columns=time_measured) 
    for fold in range(1,3):
        for lag in lags:
            if fold == 2:
                train_dir, test_dir = test_dir, train_dir
            
            train_temp_dir = train_dir + '_' + str(lag)
            test_temp_dir = test_dir + '_' + str(lag)

            train = get_batch_bispectrum2d_flat(train_temp_dir)
            test_ds = get_batch_bispectrum2d_flat(test_temp_dir)

            train_size = int(len(list(train.as_numpy_iterator()))*0.8)
            train_ds = train.take(train_size)
            val_ds = train.skip(train_size)

            log_path = os.path.join(log_dir, str(fold), str(lag))

            model = create_model_ANN_bispectrum2d_flat([i[0].shape[2] for i in train][1])
            model.summary()

            model.compile(loss='binary_crossentropy', optimizer=optimizers, metrics=['acc'])

            cpu_start = time.process_time()
            wt_start = time.time()

            history = model.fit(train_ds, epochs=epochs, validation_data=(val_ds), callbacks = myCallbacks(log_path), verbose = 0)

            wt_end = time.time()
            cpu_end = time.process_time()
            wall_time = wt_end - wt_start
            cpu_time = cpu_end - cpu_start
            training_time.loc[lag, 'CPU_Time'+ '_' + str(fold)] = cpu_time
            training_time.loc[lag, 'Wall_Time'+ '_' + str(fold)] = wall_time

            results = model.evaluate(test_ds, callbacks = myCallbacks(log_path))

            recap.loc[lag, 'train'+ '_' + str(fold)] = history.history['acc']
            recap.loc[lag, 'test'+ '_' + str(fold)] = results[1]
            recap.loc[lag, 'epoch'+ '_' + str(fold)] = len(history.history['acc'])
    log_dir = os.path.join(log_dir,'Recap')
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        print('test')
    recap.to_csv(os.path.join(log_dir,'recap.csv'))
    training_time.to_csv(os.path.join(log_dir,'Training_time.csv'))


Model: "sequential_1080"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_840 (Reshape)       (None, 61, 25, 1)         0         
                                                                 
 flatten_1080 (Flatten)      (None, 1525)              0         
                                                                 
 dense_3240 (Dense)          (None, 512)               781312    
                                                                 
 dropout_2160 (Dropout)      (None, 512)               0         
                                                                 
 dense_3241 (Dense)          (None, 256)               131328    
                                                                 
 dropout_2161 (Dropout)      (None, 256)               0         
                                                                 
 dense_3242 (Dense)          (None, 1)             